# Activitat 2: Representació Molecular amb RDKit

# **Exploració de Molècules de DrugBank**

## Bioinformàtica - UF3 

    
+ **Nom alumn@:**

*NOTA1: Per cada dia d'entrega més tard es descomptarà un 10% de la nota de la pràctica.*

*NOTA2: L'script s'ha d'executar sense errors i produir els resultats, fitxers i figures que es demanen. És aconsellable abans d'entregar reiniciar el kernel del notebook (Kernel > Restart Kernel) i comprovar que totes les cel·les del notebook s'executen sense errors una rere l'altre.* 
***

[DrugBank](https://go.drugbank.com/) és una base de dades de fàrmacs aprovats per la FDA (Food and Drug Administration, Agència del medicament dels EEUU). En aquesta activitat, treballarem amb una col·lecció de fàrmacs (`approved subset`) provinents d'aquesta base de dades. Podem descarregar el fitxer `drugbank_approved_structures.sdf` en format `SDF`(el teniu al MOODLE a l’apartat de l’Activitat 2).

L’objectiu d’aquesta activitat és realitzar un anàlisis i manipulació de les molècules utilitzant el mòdul de RDKit i Python. Recordeu  que  jupyter-lab  permet crear cel·les  en  markdown  on  podeu explicar què esteu fent i quins resultats que heu obtingut. A algunes solucions es poden arribar aplicant diferents estratègies.

**Importa els mòduls necessaris**

In [11]:
#Importar moduls
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import SDMolSupplier
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np
from PIL import Image

In [12]:
# Desactivem els warnings i errors de rdkit per una major claredat en el notebook.
from rdkit import rdBase
rdBase.DisableLog('rdApp.warning')
rdBase.DisableLog('rdApp.error')

## 1. Anàlisi i preparació de les dades

**1.a) Llegeix l'arxiu de molècules de drugbank i indica quants fàrmacs hi ha en total** [0.5 punts]

In [14]:
# Llegir l'arxiu SDF
supplier = Chem.SDMolSupplier('./drugbank_approved_structures.sdf')

# Comptar el nombre de fàrmacs
num_drugs = len([m for m in supplier if m is not None])

print(f'Hi ha un total de {num_drugs} fàrmacs en l\'arxiu.')

Hi ha un total de 2352 fàrmacs en l'arxiu.


**1.b) Mostra els camps de dades associats a la primera molècula que s'han anotat en el sdf.** [0.5 punts]

In [4]:
# Obtenir la primera molècula
first_mol = next((m for m in supplier if m is not None), None)

if first_mol is not None:
    # Mostrar els camps de dades
    for key in first_mol.GetPropNames():
        print(f'{key}: {first_mol.GetProp(key)}')
else:
    print('No s\'ha pogut carregar cap molècula.')

DATABASE_ID: DB00006
DATABASE_NAME: drugbank
SMILES: CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)[C@H](CC(O)=O)NC(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@@H]1CCCN1C(=O)[C@H](N)CC1=CC=CC=C1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC(C)C)C(O)=O
INCHI_IDENTIFIER: InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122-39-15-23-70(122)92(149)114-60(30-34-79(134)135)85(142)111-59(29-33-78(132)133)86(143)116-64(43-55-24-26-56(123)27-25-55)89(146)118-67(97(154)155)40-51(2)3)119-87(144)61(31-35-80(136)137)112-84(141)58(28-32-77(130)131)113-88(145)63(42-54-18-10-7-11-19-54)117-90(147)66(45-81(138)139)110-76(129)50-107-83(140)65(44-71(100)124)109-75(128)49-106-73(126)47-104-72(125)46-105-74(127)48-108-91(148)68-21-13-38-121(68)95(152)62(20-12-36-103-98(101)102)115-93(150)69-22-14-37-120(69)94(151)57(99)41-53-16-8-6-9-17-53/h6-11,16-19,24-2

**1.c) Afegeix un nom a cada molècula que correspongui al seu identificador de DrugBank  (*e.g.* DB00035). Assegurat que realment s'ha guardat la informació mostrant el nom de les 10 primeres molècules** [1 punt]

_Pista: 1) L'identificador es troba en una propietat de les llistades_

In [23]:
# Afegeix un nom a cada molècula i recull les 10 primeres
first_10_mols = []
for i, mol in enumerate(supplier):
    if mol is not None:
        # L'identificador de DrugBank es troba en la propietat 'DRUGBANK_ID'
        drugbank_id = mol.GetProp('DRUGBANK_ID')
        generic_name = mol.GetProp('GENERIC_NAME')
        if i < 10:
            first_10_mols.append(mol)

# Mostra el nom i la DrugBank ID de les 10 primeres molècules
for mol in first_10_mols:
    print(f'Nom: {mol.GetProp("GENERIC_NAME")}, DrugBank ID: {mol.GetProp("DRUGBANK_ID")}')

Nom: Bivalirudin, DrugBank ID: DB00006
Nom: Goserelin, DrugBank ID: DB00014
Nom: Gramicidin D, DrugBank ID: DB00027
Nom: Desmopressin, DrugBank ID: DB00035
Nom: Cetrorelix, DrugBank ID: DB00050
Nom: Daptomycin, DrugBank ID: DB00080
Nom: Ciclosporin, DrugBank ID: DB00091
Nom: Octreotide, DrugBank ID: DB00104
Nom: Abarelix, DrugBank ID: DB00106
Nom: Pyridoxal Phosphate, DrugBank ID: DB00114


## 2. Anàlisi de duplicats

**2.a) A partir dels InchiKeys comprova quants fàrmacs diferents hi ha?** [1 punt]

_Nota: Els InchiKeys van ser creats per la IUPAC i es tracta d'una cadena de caràcters capaç de representar de forma exclusiva i única una substància química. Per tant, és una forma d'eliminar molècules duplicades._

In [6]:
# Crear un conjunt per recollir els InchiKeys únics
unique_inchikeys = set()

for mol in supplier:
    if mol is not None:
        # Obtenir l'InchiKey de la molècula
        inchikey = Chem.MolToInchiKey(mol)
        # Afegir l'InchiKey al conjunt
        unique_inchikeys.add(inchikey)

# Imprimir el nombre de fàrmacs diferents
print(f'Hi ha un total de {len(unique_inchikeys)} fàrmacs diferents.')

Hi ha un total de 2351 fàrmacs diferents.


**2.b) Genera els InChiKey a partir de RDKit per a cada fàrmac i comprova si és el mateix que apareix com a camp `INCHI_KEY` en el fitxer sdf. Printa únicament els que són diferents.** [1 punt]

In [24]:
for mol in supplier:
    if mol is not None:
        generic_name = mol.GetProp('GENERIC_NAME')
        # Obtenir l'InchiKey generat per RDKit
        rdkit_inchikey = Chem.MolToInchiKey(mol)
        # Obtenir l'InchiKey del camp INCHI_KEY
        sdf_inchikey = mol.GetProp('INCHI_KEY')
        # Comprovar si els InchiKeys són diferents
        if rdkit_inchikey != sdf_inchikey:
            print(f'Diferència detectada per la molècula {mol.GetProp("GENERIC_NAME")}:')
            print(f'  InchiKey RDKit: {rdkit_inchikey}')
            print(f'  InchiKey SDF: {sdf_inchikey}\n')


Diferència detectada per la molècula Caspofungin:
  InchiKey RDKit: JYIKNQVWKBUSNH-OGZDCFRISA-N
  InchiKey SDF: JYIKNQVWKBUSNH-QWDBRQCVSA-N

Diferència detectada per la molècula Oxaliplatin:
  InchiKey RDKit: DWAFYCQODLXJNR-BNTLRKBRSA-L
  InchiKey SDF: ZROHGHOFXNOHSO-BNTLRKBRSA-L

Diferència detectada per la molècula Tetrachlorodecaoxide:
  InchiKey RDKit: HDWYUXFNUVEZOB-UHFFFAOYSA-N
  InchiKey SDF: VOWOEBADKMXUBU-UHFFFAOYSA-J

Diferència detectada per la molècula Romidepsin:
  InchiKey RDKit: OHRURASPPZQGQM-RVONLBNKSA-N
  InchiKey SDF: OHRURASPPZQGQM-QDBHDZETSA-N

Diferència detectada per la molècula Temsirolimus:
  InchiKey RDKit: CBPNZQVSJQDFBE-HGVVHKDOSA-N
  InchiKey SDF: CBPNZQVSJQDFBE-FUXHJELOSA-N

Diferència detectada per la molècula Technetium Tc-99m mebrofenin:
  InchiKey RDKit: JLJSYHOPCNWUNE-IEOVAKBOSA-N
  InchiKey SDF: JLJSYHOPCNWUNE-NLQOEHMXSA-N

Diferència detectada per la molècula Technetium Tc-99m medronate:
  InchiKey RDKit: IUPNVOAUFBLQME-LDYDCSBASA-J
  InchiKey SDF: 

Com podeu veure són casos de molècules una mica extranyes. Moltes tenen Tecneci99 un isòtop radioactiu del Tecneci que s'utilitza habitualment en mètodes de radiodiagnòstic

**2.C) Genera un fitxer `SDF` amb les molècules anteriors, passant-les a 3D i eliminant els hidrògens.** [0.75 punt]

In [8]:
# Crea un escriptor SDF
writer = Chem.SDWriter('molecules.sdf')

for mol in first_10_mols:
    # Converteix la molècula a 3D
    AllChem.EmbedMolecule(mol)
    # Elimina els hidrògens
    mol = Chem.RemoveHs(mol)
    # Escriu la molècula al fitxer SDF
    writer.write(mol)

# Tanca l'escriptor SDF
writer.close()

### 3. Càlcul de propietats moleculars

**3.a)  Ordena els fàrmacs pel seu pes molecular i mostreu per pantalla el nom, el pes molecular i l'estructura dels 5 més pesats (pes molecular més alt).** [1.5 punts]

_Pista: Resulta més fàcil fer-ho fent servir una funció lambda, però no és obligatori. També et proposem que miris la funció `sort()`_

In [9]:
res = []

for mol in supplier:
    if mol is not  None:
        nom = mol.GetProp('DRUGBANK_ID') if mol.HasProp('DRUGBANK_ID') else 'Desconegut'
        pes_molecular = Chem.Descriptors.MolWt(mol)
        smiles = Chem.MolToSmiles(mol)
        estructura = Chem.MolFromSmiles(smiles)
        res.append((nom, pes_molecular, estructura))
        
res_sorted = sorted(res, key=lambda x: x[1], reverse=True)

for r in res_sorted[:5]:
    nom, pes_molecular, estructura = r
    if mol:
        mol = Chem.AddHs(mol)
        AllChem.Compute2DCoords(mol)
        img = Draw.MolToImage(mol)
        img.show()
        print(f"nom: {nom}, Pes Molecular: {pes_molecular}\n")

nom: DB05528, Pes Molecular: 7177.247999999963

nom: DB00638, Pes Molecular: 6179.373

nom: DB08869, Pes Molecular: 5005.763999999985

nom: DB09265, Pes Molecular: 4909.513999999975

nom: DB09067, Pes Molecular: 4670.384000000007



**3.b) Exporta les imatges de les molècules anteriors a `png` i on el nom de cada arxiu correspongui al codi de DrugBank.** [0.75 punt]


In [10]:
for i, r in enumerate(res_sorted[:5]):
    nom, pes_molecular, estructura = r
    if mol:
        mol = Chem.AddHs(mol)
        AllChem.Compute2DCoords(mol)
        img = Draw.MolToImage(mol)
        img.show()

        nom_arxiu = f"{nom}_DrugBank_{i + 1}.png"
        img.save(nom_arxiu)
        print(f"Imatge exportada: {nom_arxiu}")

Imatge exportada: DB05528_DrugBank_1.png
Imatge exportada: DB00638_DrugBank_2.png
Imatge exportada: DB08869_DrugBank_3.png
Imatge exportada: DB09265_DrugBank_4.png
Imatge exportada: DB09067_DrugBank_5.png


## 4. Filtre de molècules ADME

**4.a) Filtra la base de dades de molècules per a quedar-te amb aquelles molècules que compleixin TOTES les regles de Lipinski.** [2 punts]

+ Pes molecular <= 500

+ LogP <= 5

+ Nº de donadors de pont d'H <= 5 (Hydrogen bond donors)

+ Nº d'acceptors de pont d'H <= 10 (Hydrogen bond acceptors)

Podeu fer servir l'estratègia que considereu, una forma de fer-ho és pas a pas, conservant les molècules que compleixen el primer requisit i després avaluar el següent, i així consecutivament. També podeu escollir entre calcular les propietats amb RDKit o fer servir les que es troben al SDF original, simplement que sapigueu que el nombre de molècules no serà exactament el mateix entre les dues estratègies ja que el LogP pot variar en funció de l'algoritme amb el que es calculi. 

_Nota: Amb l'alternativa de consultar les propietats del propi SDF, cal assegurar-se que el camp de consulta existeix, sinó ens donarà error._

In [18]:
def filtra_lipinski(mol):
    """
    Filtra una molécula según las reglas de Lipinski.
    Retorna True si la molécula pasa el filtro, False en caso contrario.
    """
    return (Descriptors.MolWt(mol) <= 500 and
            Descriptors.MolLogP(mol) <= 5 and
            Descriptors.NumHDonors(mol) <= 5 and
            Descriptors.NumHAcceptors(mol) <= 10)

# Crear una llista per emmagatzemar les molècules que compleixen les regles de Lipinski
molècules_lipinski = []

# Recórrer les molècules del fitxer .sdf i filtrar-les
for mol in supplier:
    if mol is not None:
        if filtra_lipinski(mol):
            molècules_lipinski.append(mol)

# Mostrar el nombre total de molècules y el nombre de molècules que compleixen las reglas de Lipinski
print(f"Nombre total de molècules: {len(supplier)}")
print(f"Nombre de molècules que compleixen les regles de Lipinski: {len(molècules_lipinski)}")

Nombre total de molècules: 2352
Nombre de molècules que compleixen les regles de Lipinski: 1724


**4.b) Exporta a `SMILES` els fàrmacs que cumpleixen les regles de Lipinski. Recorda que als fitxers SMILES s'acostuma a posar el SMILE seguit d'una tabulació i el nom de la molècula.** [1 punt] 

In [21]:
# Crear un fitxer per emmagatzemar els SMILES i noms de les molècules que compleixen les regles de Lipinski
fitxer_smiles = open('molècules_lipinski.smiles', 'w')

# Recórrer les molècules del fitxer .sdf i filtrar-les
for mol in supplier:
    if mol is not None:
        if filtra_lipinski(mol):
            # Obtenir el SMILES i el nom de la molècula
            smiles = Chem.MolToSmiles(mol)
            nom = mol.GetProp('GENERIC_NAME') if mol.HasProp('GENERIC_NAME') else 'Desconegut'
            
            # Escriure el SMILES i el nom al fitxer
            fitxer_smiles.write(f"{smiles}\t{nom}\n")

# Tancar el fitxer SMILES
fitxer_smiles.close()